# ModelExplorerMP Tutorial

In the following notebook we will see how ModelExplorerMP can be used to explore the parameter space of an agent-based model implemented in mesa. We will use a rough implementation of the Schelling model as an example. 

## Schelling Model

The Schelling model is a well-known agent-based model composed of two agent-groups with differing opinions. The model is implemented in an on-lattice grid, such that only one agent occupies a gridspace at a time and agents may only move across adjacent grid-spaces on each time step. 

Each agent has a tolerance for agents of the opposite opinion. When the fraction of neighboring agents with the opposing opinion surpasses this tolerance, the agent will move to a random empty spot on the surrounding eight patches or switch spots with a random neighbor. 

Below is a simple implementation of the Schelling model. The model is coded with the following parameters:

* **N**: the total number of agents
* **width, height**: the width and height of the grid agents occupy
* **p_split**: the fractional split between the two opinionated populations
* **zero_tolerance**: the tolerance of agents with opinion 0
* **one_tolerance**: the tolerance of agents with opinion 1

### Necessary packages

In [1]:
import mesa

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Import `Bokeh` packages for visualization of model.

In [3]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook, push_notebook
from bokeh.models import HoverTool

Import `mesa` packages for agent, grid, and model implementation.

In [4]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from scipy.spatial import distance
import matplotlib.pyplot as plt

### Model Implementation

**Agent implementation.** Each agent has a specified opinion and tolerance level. Agent move based on whether the fraction of neighbors with an opposing opinion violate the agent's tolerance.

In [5]:
class Individual(Agent):
  #age,health,job
    def __init__(self,unique_id,model,opinion): # incubation period? 
        super().__init__(unique_id,model)
        self.opinion = opinion
        
        if self.opinion == 0:
            self.tolerance = self.model.blue_tolerance
        else:
            self.tolerance = self.model.red_tolerance
  
    def step(self):
        self.move()
    
    def move(self):
        neighbors = self.model.grid.get_neighbors(self.pos,moore=True)
        fraction_opposite = sum([n.opinion != self.opinion for n in neighbors]) / len(neighbors)
        if fraction_opposite > self.tolerance:
            patches = self.model.grid.get_neighborhood(self.pos,moore=True)
            empty_patches = [self.model.grid.is_cell_empty(p) for p in patches]
            viable_patches = [p for (p,i) in zip(patches,empty_patches) if i]
            viable_patches_fraction_opposite = []
            if len(viable_patches) > 0:
                for i in viable_patches:
                    neighbors_i = self.model.grid.get_neighbors(i,moore=True)
                    fraction_opposite_i = sum([n.opinion != self.opinion for n in neighbors_i]) / len(neighbors_i)
                    viable_patches_fraction_opposite.append(fraction_opposite_i)
                new_patch = np.argmin(viable_patches_fraction_opposite)
                self.model.grid.move_agent(self, viable_patches[new_patch])
            else:
                swap_neighbor = np.random.randint(len(neighbors))
                swap_neighbor = neighbors[swap_neighbor]
                swap_pos = swap_neighbor.pos
                self.model.grid.move_agent(swap_neighbor, self.pos)
                self.model.grid.move_agent(self, swap_pos)

**Model implementation.** It is important that the model datacollector is implemented so that model data can be collected and reported by the batchrunner.

In [6]:
class SchellingModel(Model):
    def __init__(self,N,width,height,p_split,zero_tolerance,one_tolerance):
        super().__init__()
        self.num_agents = N
        self.grid = MultiGrid(width,height,True)
        self.schedule = RandomActivation(self)
        self.time = 0
        
        self.blue_tolerance = zero_tolerance
        self.red_tolerance = one_tolerance
        
        # generate N agents
        for i in range(N):
            if i <= int(p_split * N):
                a = Individual(i,self,0)
                self.schedule.add(a)
            else:
                a = Individual(i,self,1)
                self.schedule.add(a)
            
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, self.grid.find_empty())
            
        self.agent_grid = return_agent_grid(self)
        self.datacollector = DataCollector(
            model_reporters={"Agent_Grid": "agent_grid",
                                "Num_Zero_Groups":num_zero_groups,
                                "Num_One_Groups":num_one_groups},
            agent_reporters={"Location": "pos"})
   
    def step(self):
        self.schedule.step()
        self.time = self.time + 1
        self.agent_grid = return_agent_grid(self)
        self.datacollector.collect(self)

### Model Data Collection Methods

This method returns a matrix specifying which agent opinion occupies each grid space.

In [7]:
def return_agent_grid(model):
    grid = np.zeros((model.grid.width,model.grid.height))
    for x in range(model.grid.width):
        for y in range(model.grid.height):
            content = model.grid.get_cell_list_contents([(x,y)])
            if len(content) > 0:
                grid[x][y] = (content[0]).opinion
            else:
                grid[x][y] = -1
    return grid

Methods to return the number of zero and number of one groups.

In [8]:
from skimage.measure import label,regionprops,regionprops_table
def num_zero_groups(model):
    zero_grid = np.array([i == 0 for i in model.agent_grid])
    label_zero_grid = label(zero_grid,connectivity=1)
    props_zero_grid = regionprops_table(label_zero_grid, properties=('centroid','area'))
    props_zero_grid = pd.DataFrame(props_zero_grid)
    num_zero_groups = sum(props_zero_grid.area > 3)
    return num_zero_groups

def num_one_groups(model):
    one_grid = np.array([i == 1 for i in model.agent_grid])
    label_one_grid = label(one_grid,connectivity=1)
    props_one_grid = regionprops_table(label_one_grid, properties=('centroid','area'))
    props_one_grid = pd.DataFrame(props_one_grid)
    num_one_groups = sum(props_one_grid.area > 3)
    return num_one_groups

### Model Visualization Methods

In [9]:
from bokeh.transform import transform
from bokeh.models import ColumnDataSource, BasicTicker, CategoricalTicker, LinearColorMapper
output_notebook()

Loading BokehJS ...

In [10]:
def update_agents_data(agents):
    new_data = {}
    agents_positions = np.array([agent.pos for agent in agents])
    new_data['x'] = np.array([pos[0] for pos in agents_positions])
    new_data['y'] = np.array([pos[1] for pos in agents_positions])
    new_data['opinion'] = np.array([agent.opinion for agent in agents])
    new_data['id'] = np.array([agent.unique_id for agent in agents])
    new_data['tolerance'] = np.array([agent.tolerance for agent in agents])
    return new_data

In [11]:
def show_agents(agent_positions,width,height):
    p = figure(x_range=(0, width), y_range=(0, height))
    
    colors = ['blue', 'red']
    mapper = LinearColorMapper(palette=colors, 
                               low=0, 
                               high=1)

    p.circle(x='x', y='y', radius = 0.5, source=agent_positions,
       line_color=None, fill_color=transform('opinion', mapper))
    
    my_hover = HoverTool()
    my_hover.tooltips = [('id','@id'),('Opinion','@opinion'),('tolerance','@tolerance')]
    p.add_tools(my_hover)
    
    p.grid.grid_line_color = None    
    p.axis.axis_line_color = None
    p.toolbar.logo = None
    
    show(p,notebook_handle=True)

### Run the model

In [42]:
import time
width = 20
height = 20
model = SchellingModel(350, width, height,0.5,.2,.2)
agent_positions = ColumnDataSource(update_agents_data(model.schedule.agents))
show_agents(agent_positions,height,height)

for i in range(100):
    model.step()
    time.sleep(1)
    agent_positions.data = (update_agents_data(model.schedule.agents))
    push_notebook()

16
17
12
11
13
13
10
9
10
12
9
13
10
11
10
10
8
8
5
5
8


KeyboardInterrupt: 

## Model Parameter Space Exploration

Now, we will actually get to the use of ModelExplorerMP. First, import the package as follows:

In [12]:
from mesaModelAssistant import ModelExplorerMP as mp

To instantiate the ModelExplorerMP object the following parameters must be defined:
* **model_cls**: Class in which the model is defined.
* **num_cores**: the number of cores over which you would like to parallelize the batchrunning process.
* **variable_parameters**: a dictionary of the form {string: list} in which the keys define parameters to be varied in the model and the values specify the values over which to vary these parameters.
* **fixed_parameters**: a dictionary of the form {string: value} in which the key, value pairs define constant parameters in the model.
* **iterations**: the number of times each parameter combination should be run.
* **max_steps**: the number of time steps the model should be run.

In [13]:
batchrunner  = mp.ModelExplorerMP(model_cls=SchellingModel,num_cores=2,
                                                  variable_parameters = {
                                                        "p_split": [0.25,0.5,0.75],
                                                        "zero_tolerance": [0.25,0.5,0.75],
                                                        "one_tolerance": [0.25,0.5,0.75]
                                                    },
                                                   fixed_parameters = {
                                                        "N":350,
                                                        "width":20,
                                                        "height":20
                                                    },
                                                   iterations = 5,
                                                   max_steps = 100)

To run each parameter combination use the `schedule_run_all_param_combinations()` method.
The method will return all data collected by the DataCollector() object defined in model_cls for each parameter combination overtime, in the form of a list of dictionaries, in which the key is the parameter combination and iteration and the value is the returned data frame. You may analyze this data however you please :)

In [14]:
param_exploration_results = batchrunner.schedule_run_all_param_combinations()

Run ['p_split', 'zero_tolerance', 'one_tolerance'] = [0.25, 0.25, 0.25]; Iteration 0 started.
Run ['p_split', 'zero_tolerance', 'one_tolerance'] = [0.25, 0.25, 0.25]; Iteration 1 started.
Run [0.25, 0.25, 0.25]; Iteration 1 finished.
Run [0.25, 0.25, 0.25]; Iteration 0 finished.
Run ['p_split', 'zero_tolerance', 'one_tolerance'] = [0.25, 0.25, 0.25]; Iteration 2 started.
Run ['p_split', 'zero_tolerance', 'one_tolerance'] = [0.25, 0.25, 0.25]; Iteration 3 started.
Run [0.25, 0.25, 0.25]; Iteration 3 finished.
Run [0.25, 0.25, 0.25]; Iteration 2 finished.
Run ['p_split', 'zero_tolerance', 'one_tolerance'] = [0.25, 0.25, 0.25]; Iteration 4 started.
Run ['p_split', 'zero_tolerance', 'one_tolerance'] = [0.25, 0.25, 0.5]; Iteration 0 started.
Run [0.25, 0.25, 0.5]; Iteration 0 finished.
Run [0.25, 0.25, 0.25]; Iteration 4 finished.
Run ['p_split', 'zero_tolerance', 'one_tolerance'] = [0.25, 0.25, 0.5]; Iteration 1 started.
Run ['p_split', 'zero_tolerance', 'one_tolerance'] = [0.25, 0.25, 0.5

Run [0.5, 0.25, 0.75]; Iteration 4 finished.
Run [0.5, 0.25, 0.75]; Iteration 3 finished.
Run ['p_split', 'zero_tolerance', 'one_tolerance'] = [0.5, 0.5, 0.25]; Iteration 0 started.
Run ['p_split', 'zero_tolerance', 'one_tolerance'] = [0.5, 0.5, 0.25]; Iteration 1 started.
Run [0.5, 0.5, 0.25]; Iteration 0 finished.
Run [0.5, 0.5, 0.25]; Iteration 1 finished.
Run ['p_split', 'zero_tolerance', 'one_tolerance'] = [0.5, 0.5, 0.25]; Iteration 2 started.
Run ['p_split', 'zero_tolerance', 'one_tolerance'] = [0.5, 0.5, 0.25]; Iteration 3 started.
Run [0.5, 0.5, 0.25]; Iteration 2 finished.
Run [0.5, 0.5, 0.25]; Iteration 3 finished.
Run ['p_split', 'zero_tolerance', 'one_tolerance'] = [0.5, 0.5, 0.25]; Iteration 4 started.
Run ['p_split', 'zero_tolerance', 'one_tolerance'] = [0.5, 0.5, 0.5]; Iteration 0 started.
Run [0.5, 0.5, 0.5]; Iteration 0 finished.
Run [0.5, 0.5, 0.25]; Iteration 4 finished.
Run ['p_split', 'zero_tolerance', 'one_tolerance'] = [0.5, 0.5, 0.5]; Iteration 1 started.
Run [

Run [0.75, 0.5, 0.75]; Iteration 4 finished.
Run [0.75, 0.5, 0.75]; Iteration 3 finished.
Run ['p_split', 'zero_tolerance', 'one_tolerance'] = [0.75, 0.75, 0.25]; Iteration 0 started.
Run ['p_split', 'zero_tolerance', 'one_tolerance'] = [0.75, 0.75, 0.25]; Iteration 1 started.
Run [0.75, 0.75, 0.25]; Iteration 0 finished.
Run [0.75, 0.75, 0.25]; Iteration 1 finished.
Run ['p_split', 'zero_tolerance', 'one_tolerance'] = [0.75, 0.75, 0.25]; Iteration 2 started.
Run ['p_split', 'zero_tolerance', 'one_tolerance'] = [0.75, 0.75, 0.25]; Iteration 3 started.
Run [0.75, 0.75, 0.25]; Iteration 3 finished.
Run [0.75, 0.75, 0.25]; Iteration 2 finished.
Run ['p_split', 'zero_tolerance', 'one_tolerance'] = [0.75, 0.75, 0.25]; Iteration 4 started.
Run ['p_split', 'zero_tolerance', 'one_tolerance'] = [0.75, 0.75, 0.5]; Iteration 0 started.
Run [0.75, 0.75, 0.5]; Iteration 0 finished.
Run [0.75, 0.75, 0.25]; Iteration 4 finished.
Run ['p_split', 'zero_tolerance', 'one_tolerance'] = [0.75, 0.75, 0.5]; 

In [26]:
param_exploration_results

[[{"['p_split', 'zero_tolerance', 'one_tolerance']_[0.25, 0.25, 0.25]_run_0":                                            Agent_Grid  Num_Zero_Groups  \
   0   [[-1.0, 1.0, -1.0, 1.0, 1.0, 0.0, -1.0, 0.0, 1...                8   
   1   [[1.0, 1.0, -1.0, 1.0, -1.0, -1.0, 0.0, 1.0, 1...                8   
   2   [[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, -1.0, 1.0, 1.0...                7   
   3   [[1.0, 1.0, 1.0, -1.0, -1.0, 0.0, 0.0, -1.0, 1...                7   
   4   [[1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, -1.0...                8   
   ..                                                ...              ...   
   95  [[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...                5   
   96  [[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...                5   
   97  [[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...                5   
   98  [[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...                5   
   99  [[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...                4   
 